#ex1. nltk text generation

In [ ]:
import nltk
nltk.download('all')
from nltk.corpus import gutenberg
gutenberg.fileids()
import warnings
warnings.filterwarnings("ignore")

# import text of bible
bible_text = gutenberg.words('bible-kjv.txt')
nltk.bigrams(bible_text)


# generator
def generate_model(cfdist, word, num=15):
    for i in range(num):
        print(word,end=" ")
        word = cfdist[word].max()
        
bible_text = gutenberg.words('bible-kjv.txt')
bigrams = nltk.bigrams(bible_text)
cfd = nltk.ConditionalFreqDist(bigrams)

In [27]:
generate_model(cfd, 'god', num=5)

god , and the LORD 

In [28]:
generate_model(cfd, 'god', num=10)

god , and the LORD , and the LORD , 

In [29]:
generate_model(cfd, 'god', num=15)

god , and the LORD , and the LORD , and the LORD , and 

In [30]:
generate_model(cfd, 'bird', num=15)

bird , and the LORD , and the LORD , and the LORD , and 

#ex2. wn meronyms

In [31]:
from nltk.corpus import wordnet as wn
from nltk.corpus import brown

romance_words = set(brown.words(categories='romance'))
religion_words = set(brown.words(categories='religion'))

In [32]:
#  part_meronyms
def noun_part_meronym_count(words):
    meronym_list = []
    noun_counter = 0
    for word in words:
        if wn.synsets(word, wn.NOUN) != []:
            noun_counter += 1
        for synset in wn.synsets(word, wn.NOUN):
            part_meronym_list = synset.part_meronyms()
            if part_meronym_list != []:
                for part_meronym in part_meronym_list:
                    for part_meronym_lemma in part_meronym.lemma_names():
                        meronym_list.append(part_meronym_lemma)
    meronym_set = set(meronym_list)
    return((len(meronym_set),len(meronym_list)/noun_counter))

romance_result = noun_part_meronym_count(romance_words)
print('Total number of part meronym from Romance: ',romance_result[0])
print('Number of part meronym per noun from Romance: ',romance_result[1])

religion_result = noun_part_meronym_count(religion_words)
print('Total number of part meronym from Religion: ',religion_result[0])
print('Number of part meronym per noun from Religion: ',religion_result[1])

Total number of part meronym from Romance:  4674
Number of part meronym per noun from Romance:  2.0031525851197984
Total number of part meronym from Religion:  3515
Number of part meronym per noun from Religion:  1.6870474658085277


In [33]:
#  substance_meronyms
def noun_substance_meronym_count(words):
    meronym_list = []
    noun_counter = 0
    for word in words:
        if wn.synsets(word, wn.NOUN) != []:
            noun_counter += 1
        for synset in wn.synsets(word, wn.NOUN):
            substance_meronym_list = synset.substance_meronyms()
            if substance_meronym_list != []:
                for substance_meronym in substance_meronym_list:
                    for substance_meronym_lemma in substance_meronym.lemma_names():
                        meronym_list.append(substance_meronym_lemma)
    meronym_set = set(meronym_list)
    return((len(meronym_set),len(meronym_list)/noun_counter))

romance_result = noun_substance_meronym_count(romance_words)
print('Total number of substance meronym from Romance: ',romance_result[0])
print('Number of substance meronym per noun from Romance: ',romance_result[1])

religion_result = noun_substance_meronym_count(religion_words)
print('Total number of substance meronym from Religion: ',religion_result[0])
print('Number of substance meronym per noun from Religion: ',religion_result[1])

Total number of substance meronym from Romance:  166
Number of substance meronym per noun from Romance:  0.07503152585119799
Total number of substance meronym from Religion:  90
Number of substance meronym per noun from Religion:  0.032448377581120944


#ex3. wn polysemy

In [34]:
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import numpy as np

romance_words = brown.words(categories='romance')
religion_words = brown.words(categories='religion')
len(wn.synsets('dog', 'n'))

def polysemy_calculate(words):
    word_polysemy_list = []
    for word in words:
        word_polysemy_list.append(len(wn.synsets(word,'n')))
    return(np.mean(word_polysemy_list))  

In [35]:
print('average polysemy of romance noun:',polysemy_calculate(romance_words))

average polysemy of romance noun: 1.5066264888178


In [36]:
print('average polysemy of religion noun:',polysemy_calculate(religion_words))

average polysemy of religion noun: 1.6751186578339552


#ex4. wn word similarity

In [37]:
import pandas as pd
from nltk.corpus import wordnet as wn
from nltk.corpus import genesis
genesis_ic = wn.ic(genesis, False, 0.0)
simlex = pd.read_csv('SimLex-999.txt',sep='\t')
simlex

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93
...,...,...,...,...,...,...,...,...,...,...
994,join,acquire,V,2.85,2.86,2.93,2,0.00,0,0.99
995,send,attend,V,1.67,2.70,3.17,2,0.00,0,1.44
996,gather,attend,V,4.80,2.75,3.17,2,0.00,0,1.97
997,absorb,withdraw,V,2.97,3.11,3.04,2,0.00,0,1.75


In [38]:
simlex999 = simlex[['word1','word2','POS']].copy()
simlex999['simlex_score'] = simlex['SimLex999']
for score_type in ['path_score','wup_score','lch_score','jcn_score','res_score','lin_score']:
    simlex999.insert(len(simlex999.keys()),score_type,'')

simlex999 = simlex999.drop(index=(simlex999.loc[(simlex999['POS']=='A')].index))
simlex999.index = [i for i in range(len(simlex999))]

In [39]:
word1_list = simlex999['word1']
word2_list = simlex999['word2']
POS_list = simlex999['POS']

for i in range(len(simlex999)):
    #i=11
    #genesis_ic = wn.ic(genesis, False, 0.0)
    word1_symset = wn.synset(simlex999['word1'][i] + '.' + POS_list[i].lower() + '.01')
    word2_symset = wn.synset(simlex999['word2'][i] + '.' + POS_list[i].lower() + '.01')

    simlex999['path_score'][i] = word1_symset.path_similarity(word2_symset)
    simlex999['wup_score'][i] = word1_symset.wup_similarity(word2_symset)
    simlex999['lch_score'][i] = wn.lch_similarity(word1_symset, word2_symset)
    simlex999['jcn_score'][i] = wn.jcn_similarity(word1_symset, word2_symset, ic=genesis_ic)
    simlex999['res_score'][i] = wn.res_similarity(word1_symset, word2_symset, ic=genesis_ic)
    simlex999['lin_score'][i] = wn.lin_similarity(word1_symset, word2_symset, ic=genesis_ic)

In [40]:
simlex999

,word1,word2,POS,simlex_score,path_score,wup_score,lch_score,jcn_score,res_score,lin_score
0,wife,husband,N,2.30,0.333333,0.6,2.53897,0.235559,4.86268,0.696131
1,book,text,N,6.35,0.0625,0.117647,0.864997,0.0572698,-0.0830686,-0.00960604
2,groom,bride,N,3.17,0.111111,0.545455,1.44036,1e-300,1.51089,3.02179e-300
3,night,day,N,1.88,0.166667,0.545455,1.84583,0.0780871,1.98631,0.236764
4,south,north,N,2.20,0.333333,0.857143,2.53897,0.0844929,3.85964,0.394755
...,...,...,...,...,...,...,...,...,...,...
883,join,acquire,V,2.85,0.333333,0.5,2.15948,0.0921704,0,0
884,send,attend,V,1.67,0.2,0.333333,1.64866,0.0670017,0,0
885,gather,attend,V,4.80,0.25,0.4,1.8718,0.0610947,0,0
886,absorb,withdraw,V,2.97,0.142857,0.25,1.31219,1e-300,0,0


In [41]:
print('corr_simlex_path:', np.corrcoef(list(simlex999['simlex_score']), list(simlex999['path_score']))[0,1])
print('corr_simlex_wup:', np.corrcoef(list(simlex999['simlex_score']), list(simlex999['wup_score']))[0,1])
print('corr_simlex_lch:', np.corrcoef(list(simlex999['simlex_score']), list(simlex999['lch_score']))[0,1])
print('corr_simlex_jcn:', np.corrcoef(list(simlex999['simlex_score']), list(simlex999['jcn_score']))[0,1])
print('corr_simlex_res', np.corrcoef(list(simlex999['simlex_score']), list(simlex999['res_score']))[0,1])
print('corr_simlex_lin:', np.corrcoef(list(simlex999['simlex_score']), list(simlex999['lin_score']))[0,1])

corr_simlex_path: 0.43888786796125756
corr_simlex_wup: 0.3505600823618934
corr_simlex_lch: 0.43394218646174965
corr_simlex_jcn: 0.0
corr_simlex_res 0.0
corr_simlex_lin: 0.2831489470556222
